In [58]:
import numpy as np
import pandas as pd

In [62]:
dataset = pd.read_csv('datasetnew.csv', low_memory=False)

dataset = dataset.drop(['ID','name','ratings','discount_price','rand','Date','Month'], axis =1)

X_columns = [0, 1, 2, 3]  # Indices of 'main_category' and 'sub_category' columns
y_column = 4        # Index of 'Age' column

X = dataset.iloc[:, X_columns].values
y = dataset.iloc[:, y_column].values

In [63]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output = False), [0, 1, 2, 3])], remainder='passthrough')
X = ct.fit_transform(X)

In [64]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [66]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 16:] = sc.fit_transform(X_train[:, 16:])
X_test[:, 16:] = sc.transform(X_test[:, 16:])

In [67]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
#classifier = RandomForestClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [68]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 2]
 [0 0]
 [0 1]
 ...
 [1 2]
 [0 1]
 [0 3]]


In [69]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

[[11154  3034    63   983     0]
 [ 7123  4845    53   922     0]
 [ 4090  3115    85   732     0]
 [ 5783  1747    34  1149     0]
 [ 1996   504    12   337     0]]
0.360817403320701


In [70]:
import pickle
with open('age_scalar.pkl', 'wb') as scalar_file:
    pickle.dump(sc, scalar_file)
with open('age_column_transformer.pkl', 'wb') as column_transformer:
    pickle.dump(ct, column_transformer)
with open('age_label_encoder.pkl', 'wb') as label_encoder:
    pickle.dump(le, label_encoder)
with open('age_model.pkl', 'wb') as model_file:
    pickle.dump(classifier, model_file)